In [1]:
from google.colab import files
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [583]:
import pandas as pd
import numpy as np
from tensorflow import keras
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings


In [584]:
#read from csv
training_x = pd.read_csv('training_set_features.csv')
training_y = pd.read_csv('training_set_labels.csv')
test_x = pd.read_csv('test_set_features.csv')

#remove id col
training_x = training_x.iloc[:,1:]
test_x = test_x.iloc[:,1:]

#concat for label binarizer
datac = [training_x, test_x]
data = pd.concat(datac, ignore_index=True)

#binarize the categoric labels
data = pd.get_dummies(data)


#fixing the features name may not contain [] < problem
import re
regex = re.compile(r"\[|\]|<", re.IGNORECASE)
data.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in data.columns.values]
data.columns = [c.replace(' ', '_') for c in data.columns]

In [585]:
for col in data.columns:
  if np.sum((data.iloc[0:training_x.shape[0],:])[col]) == 0:
    data.drop(col)

In [517]:
# DIDN'T RUN IT FOR NOW
# Features with low correlation on target eliminated on h1n1
training_data_with_target = (data.join(training_y)).iloc[0:training_x.shape[0],:]
corrmat = training_data_with_target.corr()
corrmat = corrmat.abs()

features_to_elim = []

for i in range(corrmat.shape[0]):
  if corrmat.iloc[i][106] < 0.005:
    features_to_elim.append(corrmat.columns[i])
    print(corrmat.columns[i])

# Features with low correlation on target eliminated on seasonal
for i in range(corrmat.shape[0]):
  if corrmat.iloc[i][107] < 0.005 and not (corrmat.columns[i] in features_to_elim):
    features_to_elim.append(corrmat.columns[i])
    print(corrmat.columns[i])

features_to_elim.remove('respondent_id')
data = data.drop(labels=features_to_elim, axis=1)

household_children
race_Hispanic
race_Other_or_Multiple
hhs_geo_region_atmpeygn
hhs_geo_region_lrircsnp
census_msa_MSA,_Not_Principle__City
census_msa_MSA,_Principle_City
census_msa_Non-MSA
employment_industry_qnlwzans
employment_occupation_kldqjyjy
respondent_id
hhs_geo_region_oxchjgsf
hhs_geo_region_qufhixun
employment_industry_arjwrbjb
employment_industry_msuufmds
employment_occupation_bxpfxfdn
employment_occupation_emcorrxb
employment_occupation_hodpvpew
employment_occupation_xzmlyyjv


In [586]:
# One of the highly correlated features eliminated
corrmat = corrmat.abs()

for i in range(corrmat.shape[0]):
  for j in range(corrmat.shape[1]):
    if i != j and corrmat.iloc[i][j] > 0.8:
      print("Feature1: ", corrmat.columns[i], "Feature2:", corrmat.columns[j])

columns_to_drop = ["sex_Female", "marital_status_Not_Married", "rent_or_own_Own", "employment_occupation_dcjcmpih"]
data = data.drop(labels=columns_to_drop, axis=1)

Feature1:  sex_Female Feature2: sex_Male
Feature1:  sex_Male Feature2: sex_Female
Feature1:  marital_status_Married Feature2: marital_status_Not_Married
Feature1:  marital_status_Not_Married Feature2: marital_status_Married
Feature1:  rent_or_own_Own Feature2: rent_or_own_Rent
Feature1:  rent_or_own_Rent Feature2: rent_or_own_Own
Feature1:  employment_status_Employed Feature2: employment_status_Not_in_Labor_Force
Feature1:  employment_status_Not_in_Labor_Force Feature2: employment_status_Employed
Feature1:  employment_industry_haxffmxo Feature2: employment_occupation_dcjcmpih
Feature1:  employment_occupation_dcjcmpih Feature2: employment_industry_haxffmxo


In [182]:
# DIDN'T RUN IT FOR NOW
#DEALING WITH NULLS?
for column in data.columns:
    data[column].interpolate(method='linear', direction = 'forward', inplace=True)

In [183]:
# DIDN'T RUN IT FOR NOW
#PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=90)
data = pd.DataFrame(pca.fit_transform(data))

In [587]:
#split the data back to test and train 
training_x = data.iloc[0:training_x.shape[0],:]
test_x = data.iloc[training_x.shape[0]:]

In [588]:
#split to validation and training set
from sklearn.model_selection import train_test_split
perc=0.2
X_train, X_test, y_train, y_test = train_test_split(training_x, training_y, test_size=perc)

In [276]:
# DIDN'T RUN IT FOR NOW
#standard scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns= X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns= X_test.columns)
test_x = pd.DataFrame(scaler.fit_transform(test_x), columns=test_x.columns)

In [589]:
#minMax Scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns= X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns= X_test.columns)
test_x = pd.DataFrame(scaler.fit_transform(test_x), columns=test_x.columns)

In [561]:
from xgboost import XGBClassifier

model_h1n1 = XGBClassifier(
    max_depth=2,
    n_estimators=100000,
    subsample=1,
    learning_rate=0.1,
    seed=42)

In [600]:
model_seasonal = XGBClassifier(
    max_depth=3,
    n_estimators=100000,
    subsample=0.9,
    learning_rate=0.1,
    reg_alpha=0.2,
    #min_child_weight=3,
    seed=42)

In [563]:
model_h1n1.fit(
    X_train, 
    y_train['h1n1_vaccine'], 
    eval_metric="auc", 
    eval_set=[(X_train, y_train['h1n1_vaccine']), (X_test, y_test['h1n1_vaccine'])], 
    verbose=True, 
    early_stopping_rounds = 1000)


[0]	validation_0-auc:0.762905	validation_1-auc:0.774518
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 1000 rounds.
[1]	validation_0-auc:0.793372	validation_1-auc:0.80369
[2]	validation_0-auc:0.793372	validation_1-auc:0.80369
[3]	validation_0-auc:0.793372	validation_1-auc:0.80369
[4]	validation_0-auc:0.818248	validation_1-auc:0.83002
[5]	validation_0-auc:0.812739	validation_1-auc:0.82528
[6]	validation_0-auc:0.814682	validation_1-auc:0.827331
[7]	validation_0-auc:0.818482	validation_1-auc:0.830687
[8]	validation_0-auc:0.822013	validation_1-auc:0.834624
[9]	validation_0-auc:0.822449	validation_1-auc:0.834818
[10]	validation_0-auc:0.824538	validation_1-auc:0.836285
[11]	validation_0-auc:0.826548	validation_1-auc:0.837955
[12]	validation_0-auc:0.828467	validation_1-auc:0.839938
[13]	validation_0-auc:0.827526	validation_1-auc:0.839699
[14]	validation_0-auc:0.834776	validation_1-auc:0.846457
[

XGBClassifier(max_depth=2, n_estimators=100000, seed=42)

In [601]:
model_seasonal.fit(
    X_train, 
    y_train['seasonal_vaccine'], 
    eval_metric="auc", 
    eval_set=[(X_train, y_train['seasonal_vaccine']), (X_test, y_test['seasonal_vaccine'])], 
    verbose=True, 
    early_stopping_rounds = 1000)


[0]	validation_0-auc:0.793654	validation_1-auc:0.800806
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 1000 rounds.
[1]	validation_0-auc:0.803282	validation_1-auc:0.811168
[2]	validation_0-auc:0.811875	validation_1-auc:0.818738
[3]	validation_0-auc:0.812275	validation_1-auc:0.819138
[4]	validation_0-auc:0.815064	validation_1-auc:0.82132
[5]	validation_0-auc:0.821477	validation_1-auc:0.826736
[6]	validation_0-auc:0.826918	validation_1-auc:0.830863
[7]	validation_0-auc:0.826899	validation_1-auc:0.830462
[8]	validation_0-auc:0.827973	validation_1-auc:0.831158
[9]	validation_0-auc:0.830411	validation_1-auc:0.833462
[10]	validation_0-auc:0.833674	validation_1-auc:0.835978
[11]	validation_0-auc:0.834426	validation_1-auc:0.836906
[12]	validation_0-auc:0.835899	validation_1-auc:0.838317
[13]	validation_0-auc:0.837861	validation_1-auc:0.839999
[14]	validation_0-auc:0.839224	validation_1-auc:0.8406

XGBClassifier(n_estimators=100000, reg_alpha=0.2, seed=42, subsample=0.9)

In [602]:
preds_h1n1 = model_h1n1.predict_proba(test_x)
preds_seasonal = model_seasonal.predict_proba(test_x)
submission = pd.read_csv('submission_format.csv')
submission['h1n1_vaccine'] = 1-preds_h1n1
submission['seasonal_vaccine'] = 1-preds_seasonal

In [603]:
submission.head(100)

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,26707,0.080843,0.316600
1,26708,0.018578,0.030835
2,26709,0.160334,0.667646
3,26710,0.628014,0.884254
4,26711,0.338612,0.430948
...,...,...,...
95,26802,0.081599,0.174523
96,26803,0.033626,0.118338
97,26804,0.044023,0.066368
98,26805,0.061933,0.133368


In [604]:
submission.to_csv('submission_9.csv', index=False)

In [166]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(verbose=1, learning_rate=0.001, max_depth=50)
clf.fit(X_train, y_train['seasonal_vaccine'])

ValueError: ignored